# 🎙️ Amir Voice Test - Ultimate TTS Playground

This notebook consolidates all Text-to-Speech engines for the **Su6i Yar** project.

## 🛠️ Prerequisites & Setup
Before running this notebook, ensure you have the following installed:

### 1. Python Libraries
Run the following command in your terminal:
```bash
pip install edge-tts sherpa-onnx soundfile numpy requests onnx
```

### 2. System Tools (FFmpeg)
Required for audio conversion (WAV -> MP3).
- **Mac:** `brew install ffmpeg`
- **Linux:** `sudo apt install ffmpeg`
- **Windows:** Download from ffmpeg.org and add to PATH.

### 3. Local Model (Sherpa)
Ensure the model file exists at `../../models/fa_IR-mana-medium-fixed.onnx`.

---


## 1. Setup & Imports
Run this cell first to load necessary libraries.

In [9]:
import os
import sys
import time
import wave
import struct
import numpy as np
import soundfile as sf
import json
import requests
from IPython.display import Audio, display, Markdown

# Try importing Sherpa (Local Engine)
try:
    import sherpa_onnx
    SHERPA_AVAILABLE = True
    print("✅ sherpa-onnx found.")
except ImportError:
    SHERPA_AVAILABLE = False
    print("❌ sherpa-onnx not found. (Local TTS will be disabled)")

# Constants for Local Model
MODEL_PATH = "../../models/fa_IR-mana-medium-fixed.onnx"
ORIGINAL_CONFIG = "../../models/fa_IR-mana-medium.onnx.json"
TOKENS_PATH = "../../models/tokens.txt"
ESPEAK_DATA = "/opt/homebrew/share/espeak-ng-data"

# 🌍 Test Sentences
TEST_PHRASES = {
    "fa": "سلام امیرجان. این تست صدای مانا روی کامپیوتر شماست. سرعت را ببینید!",
    "en": "Hello Amir! This is a test of the TTS model speaking English.",
    "fr": "Bonjour! C'est un test du modèle TTS parlant français.",
    "ko": "안녕하세요! 이것은 페르시아어 모델이 한국어를 말하는 테스트입니다."
}

def convert_wav_to_mp3(wav_path, mp3_path):
    """Helper to convert WAV to MP3 using ffmpeg for user request"""
    if os.path.exists(mp3_path):
        os.remove(mp3_path)
    cmd = f'ffmpeg -y -v error -i "{wav_path}" -acodec libmp3lame -q:a 2 "{mp3_path}"'
    os.system(cmd)
    return mp3_path

✅ sherpa-onnx found.


## 2. 🧠 Engine: EdgeTTS (Farid)
**Model:** Microsoft Farid (Online)  
**Quality:** Standard, slightly robotic

In [10]:
def run_edgetts_tests():
    print("--- 🧠 Running Microsoft EdgeTTS Tests (All Voices) ---")
    
    # Voices to test per language
    VOICES = {
        "fa": ["fa-IR-FaridNeural", "fa-IR-DilaraNeural"],
        "en": ["en-US-JennyNeural", "en-US-GuyNeural"],
        "fr": ["fr-FR-DeniseNeural", "fr-FR-HenriNeural"],
        "ko": ["ko-KR-SunHiNeural", "ko-KR-InJoonNeural"]
    }
    
    for lang, text in TEST_PHRASES.items():
        display(Markdown(f"### 🏳️ {lang.upper()}"))
        target_voices = VOICES.get(lang, [])
        
        for voice in target_voices:
            display(Markdown(f"**🗣️ Voice:** `{voice}`"))
            mp3_filename = f"edge_{lang}_{voice}.mp3"
            
            cmd = f'edge-tts --text "{text}" --voice {voice} --write-media {mp3_filename}'
            
            start = time.time()
            res = os.system(cmd)
            if res == 0:
                 print(f"✅ Generated in {time.time()-start:.2f}s")
                 display(Audio(mp3_filename, autoplay=False))
            else:
                print(f"❌ Failed for {voice}")
            print("-" * 10)
        print("=" * 30)

run_edgetts_tests()

--- 🧠 Running Microsoft EdgeTTS Tests (All Voices) ---


### 🏳️ FA

**🗣️ Voice:** `fa-IR-FaridNeural`

✅ Generated in 2.46s


----------


...

✅ Generated in 0.45s


----------


**🗣️ Voice:** `ko-KR-InJoonNeural`

✅ Generated in 0.95s


----------


## 3. 🧪 Engine: Local Piper (Sherpa-ONNX)
**Model:** Mana (Medium)  
**Speed:** Real-time on CPU (M1/M2)  
**Quality:** High

In [11]:
# Initialize Sherpa Engine
if SHERPA_AVAILABLE and os.path.exists(MODEL_PATH):
    print("🚀 Initializing Sherpa-ONNX...")
    
    # Generate tokens if missing
    if not os.path.exists(TOKENS_PATH) and os.path.exists(ORIGINAL_CONFIG):
        with open(ORIGINAL_CONFIG, "r", encoding="utf-8") as f:
            data = json.load(f)
        id_map = data.get("phoneme_id_map", {})
        with open(TOKENS_PATH, "w", encoding="utf-8") as f:
            for symbol, ids in id_map.items():
                if ids:
                    f.write(f"{symbol} {ids[0]}\n")
        print("✅ tokens.txt generated.")

    try:
        config = sherpa_onnx.OfflineTtsConfig(
            model=sherpa_onnx.OfflineTtsModelConfig(
                vits=sherpa_onnx.OfflineTtsVitsModelConfig(
                    model=MODEL_PATH,
                    tokens=TOKENS_PATH,
                    data_dir=ESPEAK_DATA,
                    noise_scale=0.667,
                    length_scale=1.0,
                    noise_scale_w=0.8,
                ),
                provider="cpu",
                num_threads=1,
                debug=False
            )
        )
        tts_sherpa = sherpa_onnx.OfflineTts(config)
        print("✅ Sherpa Engine Ready!")
    except Exception as e:
        print(f"❌ Initialization Failed: {e}")
        tts_sherpa = None
else:
    print("⚠️ Sherpa engine skipped (missing files or lib).")
    tts_sherpa = None

def run_sherpa_tests():
    if not tts_sherpa:
        print("❌ Engine not loaded.")
        return
        
    print("--- 🧪 Running Sherpa Multi-Lang Tests ---")
    for lang, text in TEST_PHRASES.items():
        display(Markdown(f"**{lang.upper()}:** `{text}`"))
        
        start = time.time()
        try:
            audio = tts_sherpa.generate(text, sid=0, speed=1.0)
            duration = time.time() - start
            
            # Save as WAV first
            wav_filename = f"sherpa_{lang}.wav"
            mp3_filename = f"sherpa_{lang}.mp3"
            
            if len(audio.samples) > 0:
                sf.write(wav_filename, audio.samples, audio.sample_rate)
                
                # Convert to MP3
                convert_wav_to_mp3(wav_filename, mp3_filename)
                
                print(f"⚡ Generated in {duration:.3f}s -> {mp3_filename}")
                display(Audio(mp3_filename, autoplay=False))
            else:
                print(f"⚠️ No audio generated for {lang}")
                
        except Exception as e:
            print(f"❌ Failed for {lang}: {e}")
        
        print("-" * 30)

run_sherpa_tests()

🚀 Initializing Sherpa-ONNX...
✅ Sherpa Engine Ready!
--- 🧪 Running Sherpa Multi-Lang Tests ---


**FA:** `سلام امیرجان. این تست صدای مانا روی کامپیوتر شماست. سرعت را ببینید!`

⚡ Generated in 0.366s -> sherpa_fa.mp3


------------------------------


**EN:** `Hello Amir! This is a test of the TTS model speaking English.`

...

⚡ Generated in 0.240s -> sherpa_fr.mp3


------------------------------


**KO:** `안녕하세요! 이것은 페르시아어 모델이 한국어를 말하는 테스트입니다.`

⚡ Generated in 0.276s -> sherpa_ko.mp3


------------------------------


## 4. ☁️ Engine: Datacula (Amir)
**Model:** Amir  
**Type:** Online API  
**Latency:** Depends on network

In [13]:
def run_datacula_tests():
    print("--- ☁️ Running Datacula Multi-Lang Tests ---")
    url = "https://tts.datacula.com/api/tts"
    
    for lang, text in TEST_PHRASES.items():
        display(Markdown(f"**{lang.upper()}:** `{text}`"))
        params = {"text": text, "model_name": "امیر"}
        
        start = time.time()
        try:
            resp = requests.get(url, params=params, timeout=10)
            if resp.status_code == 200:
                print(f"✅ Generated in {time.time()-start:.2f}s")
                
                # Save
                wav_filename = f"datacula_{lang}.wav"
                mp3_filename = f"datacula_{lang}.mp3"
                
                with open(wav_filename, "wb") as f:
                    f.write(resp.content)
                
                convert_wav_to_mp3(wav_filename, mp3_filename)
                display(Audio(mp3_filename, autoplay=False))
            else:
                print(f"❌ Failed: {resp.status_code}")
        except Exception as e:
            print(f"❌ Error: {e}")
        print("-" * 30)

run_datacula_tests() # to run (requires internet & API key validity)

--- ☁️ Running Datacula Multi-Lang Tests ---


**FA:** `سلام امیرجان. این تست صدای مانا روی کامپیوتر شماست. سرعت را ببینید!`

✅ Generated in 4.52s


------------------------------


**EN:** `Hello Amir! This is a test of the TTS model speaking English.`

...

✅ Generated in 3.55s


------------------------------


**KO:** `안녕하세요! 이것은 페르시아어 모델이 한국어를 말하는 테스트입니다.`

✅ Generated in 3.58s


------------------------------


## 5. ☁️ Engine: ElevenLabs
**Model:** Adam  
**Type:** Online API  
**Latency:** Depends on network

---

In [ ]:
import os
import time
import json
import urllib.request
from dotenv import load_dotenv
from IPython.display import Audio, display, Markdown

# Load Environment
load_dotenv()
API_KEY = os.getenv("ELEVENLABS_API_KEY")

# 5 Persian Voices (Discovered via API)
VOICES = {
    "Adrian": "BognUUMX6W1qmZKB2TOw",
    "Noushin": "NZiuR1C6kVMSWHG27sIM",
    "Amir": "PleK417YVMP2SUWm8Btb",
    "Nazy": "WwAjIyMBDBNl1dvId9Xe",
    "Shahram": "rNb3hdSf0n4ROIbYC8Bl"
}

# Test Sentence
text_fa = "سلام امیرجان. این تست صدای مانا روی کامپیوتر شماست. سرعت را ببینید!"

print("--- 🌟 Running ElevenLabs Persian Voice Tests ---")

def generate_elevenlabs_audio(voice_name, voice_id, text):
    url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"
    headers = {
        "xi-api-key": API_KEY,
        "Content-Type": "application/json"
    }
    data = {
        "text": text,
        "model_id": "eleven_multilingual_v2",
        "voice_settings": {"stability": 0.5, "similarity_boost": 0.75}
    }
    
    start = time.time()
    try:
        req = urllib.request.Request(url, data=json.dumps(data).encode('utf-8'), headers=headers)
        with urllib.request.urlopen(req) as response:
            audio_data = response.read()
            
        filename = f"elevenlabs_{voice_name}.mp3"
        with open(filename, "wb") as f:
            f.write(audio_data)
            
        duration = time.time() - start
        print(f"✅ {voice_name}: Generated in {duration:.2f}s")
        display(Audio(filename))
    except Exception as e:
        print(f"❌ {voice_name} Failed: {e}")

if API_KEY:
    for name, vid in VOICES.items():
        display(Markdown(f"### 🗣️ {name}"))
        generate_elevenlabs_audio(name, vid, text_fa)
        time.sleep(1) # Respect rate limits
else:
    print("❌ No ELEVENLABS_API_KEY found. Check your .env file.")

--- در حال تولید متن ---

    سلام! من صدای نسل جدید هوش مصنوعی هستم. 
    کیفیت من دقیقاً همون چیزیه که برای پادکست‌ها و ویدیوهای حرفه‌ای دنبالش بودی. 
    امیدوارم از این خروجی لذت ببری!
    
------------------------
🎙️ در حال تولید صدای استودیویی با ElevenLabs...
✅ ایولا! فایل با موفقیت در 'elevenlabs_final.mp3' ذخیره شد.
